In [ ]:
# #############################################################################
# Importing necessary libraries for creating model

from keras.models import model_from_json
from keras.models import load_model
import os
import pandas as pd
import numpy as np
import keras
#import dlib #Importing library
import cv2 # importing open CV library
import face_recognition # importing face recognition library
import imutils # Package of convenience functions

# #############################################################################
# Reading the trained model from the folder or server
import time
start = time.time()

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model.h5")

from PIL import Image
from skimage import io
import matplotlib.pyplot as plt


def detect_faces(image):                    # Create a face detector

    face_detector = dlib.get_frontal_face_detector()
    
    detected_faces = face_detector(image, 1)    # Run detector and get bounding boxes of the faces on image
    face_frames = [(x.left(), x.top(),
                    x.right(), x.bottom()) for x in detected_faces]

    return face_frames


def resize_im(image, hight, width):          # Resizing the image
    resized_im = image.resize((hight, width))
    return resized_im

def reshape(numpy_array):            # Reshape the image
    d = np.stack(numpy_array)
    reshaped = d.reshape(1, d.size)
    return reshaped

import glob #globbing utility.
import numpy as np
f = open("age.csv", "a")
path = 'C:\\Users\\rahmanar\\scikit_learn_data\\lfw_home\\*.jpg'    # selecting the path of the images
for file in glob.glob(path):
    a= cv2.imread(file)
    detected_faces = detect_faces(a)
    for n, face_rect in enumerate(detected_faces):
        fa = Image.fromarray(a).crop(face_rect)
        s = np.array(resize_im(fa, 100,100))
        cv2.namedWindow('path', cv2.WINDOW_NORMAL)
        cv2.imshow('path', s)
        c = cv2.cvtColor(s, cv2.COLOR_BGR2GRAY)
        re_data = reshape(c)
        np.savetxt(f, re_data, delimiter=",", fmt = '%d')
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
# #############################################################################
# Reading the test data from the folder or server and processed

data = pd.read_csv('storage.csv', header = None)
df_x = data.iloc[:,0:].values.reshape(len(data),100,100,1)
#data.close()
f_x = np.array(df_x)

# #############################################################################
# evaluate loaded model on test data

loaded_model.compile(loss='categorical_crossentropy',optimizer = 'adadelta', metrics = ['accuracy'])

# #############################################################################
# Predicting gender and reporting the precision, recall F1-score and accuracy

from sklearn.metrics import classification_report, confusion_matrix

y_pred = loaded_model.predict_classes(f_x)

prob = np.asmatrix(loaded_model.predict(f_x))
g_label = ["Female", "Male"]
def result(d):
    if d[0,0] > d[0,1]:
        return d[0,0], g_label[0]
    if d[0,0] < d[0,1]:
        return d[0,1], g_label[1]
gh = result(prob)
print("Confidence:", gh[0] , "Gender:", gh[1] )


In [ ]:
# #############################################################################
# 
#Full model together
#
# #############################################################################


In [ ]:
# #############################################################################
# Importing necessary libraries for creating model

from keras.models import model_from_json
from keras.models import load_model
import os
import pandas as pd
import numpy as np
import keras
import cv2 # importing open CV library
import face_recognition # importing face recognition library
import imutils # Package of convenience functions
import dlib   # For frontal face identity
from PIL import Image
from skimage import io
import matplotlib.pyplot as plt
import csv
import requests

# #############################################################################
# Reading the trained model from the folder or server

json_file = open('model.json', 'r')     # Loading the jason file
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model.h5")    # Loading the weights


def detect_faces(image):                    # Create a face detector
    face_detector = dlib.get_frontal_face_detector()
    detected_faces = face_detector(image, 1)    # Run detector and get bounding boxes of the faces on image
    face_frames = [(x.left(), x.top(),
                    x.right(), x.bottom()) for x in detected_faces]  # To identify the frame size
    return face_frames


def resize_im(image, hight, width):          # Resizing the image
    resized_im = image.resize((hight, width))
    return resized_im


def reshape(numpy_array):            # Reshape the image
    d = np.stack(numpy_array)
    reshaped = d.reshape(1, d.size)
    return reshaped
 
    
with open('H:\\privat\\face_female.csv') as csvfile:
    csvrows = csv.reader(csvfile, delimiter=',', quotechar='"')     # Reading websites from csv file
    for row in csvrows:
        filename = str(2)+ ".jpg"    # Putting the file name and the image formate
        url = row[1]   # Image url in the file
        result = requests.get(url, stream=True)   #Sending request to the websites
        if result.status_code == 200:
            image = result.raw.read()
            open(filename,"wb").write(image)    # Writing the images in the folder
            import glob       #globbing utility.
            import numpy as np
            f = open("storage.csv", "w")
            path = '*.jpg'    # selecting the path of the images
            for file in glob.glob(path):
                a= cv2.imread(file)
                detected_faces = detect_faces(a)   # Detecting the face or faces 
                for n, face_rect in enumerate(detected_faces):
                    fa = Image.fromarray(a).crop(face_rect)   # Cropping the face or faces
                    s = np.array(resize_im(fa, 100,100))
                    #cv2.namedWindow('path', cv2.WINDOW_NORMAL)
                    #cv2.imshow('path', s)
                    c = cv2.cvtColor(s, cv2.COLOR_BGR2GRAY)    # Converting the image
                    re_data = reshape(c)
                    np.savetxt(f, re_data, delimiter=",", fmt = '%d')  # Save the converted image data
                    cv2.waitKey(0)
                    cv2.destroyAllWindows()    # Need to kill the image window
        
                    # #############################################################################
                    # Reading the test data from the folder or server and processed

                    data = pd.read_csv('storage.csv', header = None)
                    df_x = data.iloc[:,0:].values.reshape(len(data),100,100,1)   # Creating the requird formate
                    #data.close()
                    f_x = np.array(df_x)  # Need to convert as a array

                    # #############################################################################
                    # evaluate loaded model on test data

                    loaded_model.compile(loss='categorical_crossentropy',optimizer = 'adadelta', metrics = ['accuracy'])

                    # #############################################################################
                    # Predicting gender and reporting the precision, recall F1-score and accuracy

                    from sklearn.metrics import classification_report, confusion_matrix

                    y_pred = loaded_model.predict_classes(f_x)    # Predicting the gender

                    prob = np.asmatrix(loaded_model.predict(f_x))   # Calculating the probability
                    g_label = ["Female", "Male"]
                    def result(d):           # Finding the requird results
                        if d[0,0] > d[0,1]:
                            return d[0,0], g_label[0]
                        if d[0,0] < d[0,1]:
                            return d[0,1], g_label[1]
                    gh = result(prob)
                    files = np.array((row[0], url, gh[0], gh[1]), dtype = str)    # Must be string data
                    d = open("gesis_female.csv", "a")
                    re_dat = reshape(files)     # Need to reshape again
                    np.savetxt(d, re_dat, delimiter=",", fmt='%s')    # Saving the output
                    print("Confidence:", gh[0] , "Gender:", gh[1] )
        print(url)
end = time.time()
print("Calculation Time:", (end - start), "seconds")       